In [ ]:
!pip install clearml

In [ ]:
# Импорт необходимых библиотек для реализации CycleGAN

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import zipfile
from torch.optim.lr_scheduler import StepLR 
from clearml import Task, Logger

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=''
%env CLEARML_API_SECRET_KEY=''

In [ ]:
# Инициализируем удаленную таску
task = Task.init(
        project_name="CycleGAN_Monet",
        task_name="CycleGAN_150_epoch",
        task_type="testing",
)
task:Task

logger = Logger.current_logger()

In [ ]:
# Фиксация seed для воспроизводимости результатов

def fix_seeds(seed: int):
    np.random.seed(seed)  # Для numpy операций
    random.seed(seed)  # Для встроенного random, который используется в DataLoader
    torch.manual_seed(seed)  # Для CPU операций PyTorch
    torch.cuda.manual_seed(seed)  # Для GPU операций 

fix_seeds(0)

In [ ]:
# Создание Residual блок - ключевой компонент для глубоких сетей, который позволяет избежать проблемы затухающих градиентов через skip connection

class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),  
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels), 
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels)
        )

    def forward(self, x):
        return x + self.conv_block(x)

In [ ]:
# Создание генератора с архитектурой encoder-transformer-decoder

class Generator(nn.Module):
    def __init__(self, input_channels=3, output_channels=3, n_residual_blocks=11):
        super(Generator, self).__init__()
        
        # Encoder - начальный слой извлечения признаков
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_channels, 64, 7), 
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]
        
        # Downsampling - уменьшение пространственного разрешения с увеличением каналов
        in_channels = 64
        for _ in range(2):
            out_channels = in_channels * 2
            model += [
                nn.Conv2d(in_channels, out_channels, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ]
            in_channels = out_channels
        
        # Transformer - обработка в пространстве признаков через residual блоки
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_channels)]
        
        # Upsampling - восстановление исходного разрешения
        for _ in range(2):
            out_channels = in_channels // 2
            model += [
                nn.ConvTranspose2d(in_channels, out_channels, 3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ]
            in_channels = out_channels
        
        # Decoder - финальный слой для получения RGB изображения
        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(64, output_channels, 7),
            nn.Tanh()
        ]
        
        self.model = nn.Sequential(*model)
    
    def forward(self, x):
        return self.model(x)


In [ ]:
# Создание PatchGAN Discriminator - классифицирует не все изображение целиком, а отдельные патчи (небольшие области изображения).

class Discriminator(nn.Module):
    def __init__(self, input_channels=3):
        super(Discriminator, self).__init__()
        
        # Вспомогательная функция для создания блоков дискриминатора
        def discriminator_block(in_channels, out_channels, normalize=True):
            layers = [nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2, inplace=True))  # LeakyReLU для стабильности обучения
            return layers
        
        # Архитектура - серия сверточных слоев со stride=2 
        self.model = nn.Sequential(
            *discriminator_block(input_channels, 64, normalize=False),
            *discriminator_block(64, 128),   
            *discriminator_block(128, 256), 
            *discriminator_block(256, 512), 
            nn.ZeroPad2d((1, 0, 1, 0)),  # Асимметричный padding для корректной свертки
            nn.Conv2d(512, 1, 4, padding=1)  # Карта вероятностей на выходе  
        )
    
    def forward(self, img):
        return self.model(img)

In [ ]:
# Создание Датасета

class MonetPhotoDataset(torch.utils.data.Dataset):
    def __init__(self, monet_dir, photo_dir, transform=None):
        self.monet_dir = monet_dir
        self.photo_dir = photo_dir
        self.transform = transform
        
        # Загрузка списков файлов
        self.monet_files = [f for f in os.listdir(monet_dir) if f.endswith('.jpg')]
        all_photo_files = [f for f in os.listdir(photo_dir) if f.endswith('.jpg')]
        self.photo_files = random.sample(all_photo_files, k=1000) # Ограничиваем количество фото для ускорения обучения
        
    def __len__(self):
        # Определение длины бОльшим датасетом
        return max(len(self.monet_files), len(self.photo_files))
    
    def __getitem__(self, idx):
        # Используется random.choice вместо индексации по idx, чтобы обеспечить случайное сопоставление
        monet_file = random.choice(self.monet_files)
        photo_file = random.choice(self.photo_files)
        
        monet_path = os.path.join(self.monet_dir, monet_file)
        photo_path = os.path.join(self.photo_dir, photo_file)
        
        # Загрузка и конвертация в RGB
        monet_img = Image.open(monet_path).convert('RGB')
        photo_img = Image.open(photo_path).convert('RGB')
        
        # Подготовка изображений
        if self.transform:
            monet_img = self.transform(monet_img)
            photo_img = self.transform(photo_img)
            
        return photo_img, monet_img

In [ ]:
# Создание Image Pool - для стабилизации обучения GAN

class ImagePool:
    def __init__(self, pool_size):
        self.pool_size = pool_size
        if self.pool_size > 0:
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        # Возвращение изображения из истории вместо только текущих
        if self.pool_size == 0:
            return images
        return_images = []
        for image in images:
            image = torch.unsqueeze(image.data, 0)
            if self.num_imgs < self.pool_size:
                # Заполнение pool до максимального размера
                self.num_imgs = self.num_imgs + 1
                self.images.append(image)
                return_images.append(image)
            else:
                # Pool заполнен: с вероятностью 50% возвращается старое изображение
                p = random.uniform(0, 1)
                if p > 0.5:
                    # Замена случайного изображения в pool и возвращение старого
                    random_id = random.randint(0, self.pool_size - 1)
                    tmp = self.images[random_id].clone()
                    self.images[random_id] = image
                    return_images.append(tmp)
                else:
                    # Возвращение текущего изображения
                    return_images.append(image)
        return_images = torch.cat(return_images, 0)
        return return_images


In [ ]:
# Создание основного класса CycleGAN, объединяющего все компоненты

class CycleGAN:
    def __init__(self, device='cuda'):
        self.device = device
        
        # Инициализация двух генераторов и двух дискриминаторов
        self.G_photo_to_monet = Generator().to(device)
        self.G_monet_to_photo = Generator().to(device)
        self.D_photo = Discriminator().to(device)
        self.D_monet = Discriminator().to(device)
        
        # Оптимизаторы Adam для генераторов и дискриминаторов
        self.optimizer_G = optim.Adam(
            list(self.G_photo_to_monet.parameters()) + list(self.G_monet_to_photo.parameters()),
            lr=0.0002, betas=(0.5, 0.999)
        )
        self.optimizer_D_photo = optim.Adam(self.D_photo.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.optimizer_D_monet = optim.Adam(self.D_monet.parameters(), lr=0.0002, betas=(0.5, 0.999))
        
        # Schedulers для постепенного снижения learning rate для улучшения сходимости
        self.scheduler_G = StepLR(self.optimizer_G, step_size=50, gamma=0.5)
        self.scheduler_D_photo = StepLR(self.optimizer_D_photo, step_size=50, gamma=0.5)  
        self.scheduler_D_monet = StepLR(self.optimizer_D_monet, step_size=50, gamma=0.5)

        # Функции потерь для разных компонентов
        self.criterion_gan = nn.MSELoss()  # Для adversarial loss (насколько хорошо генератор обманывает дискриминатор)
        self.criterion_cycle = nn.L1Loss()  # Для cycle consistency loss (гарантия того, что преобразование обратимо)
        self.criterion_identity = nn.L1Loss()  # Для identity loss (сохранение цветовой гаммы и предотвращение излишних изменений)
        
        # Веса для компонентов функции потерь
        self.lambda_cycle = 10.0  # Cycle consistency - заставляет модель сохранять содержание изображения при преобразовании
        self.lambda_identity = 0.5  # Identity loss - помогает сохранить цветовую гамму оригинала
        
        # Image pools для стабилизации обучения дискриминаторов - используют историю сгенерированных изображений вместо только текущих
        self.fake_photo_buffer = ImagePool(50) # Хранит 50 последних сгенерированных изображений
        self.fake_monet_buffer = ImagePool(50)
    
    def set_input(self, real_photo, real_monet):
        # Перемещение батчей данных на устройство (CPU/GPU)
        self.real_photo = real_photo.to(self.device)
        self.real_monet = real_monet.to(self.device)
    
    def forward(self):
        # Forward pass через оба генератора
        self.fake_monet = self.G_photo_to_monet(self.real_photo) # photo - fake_monet - reconstructed_photo
        self.rec_photo = self.G_monet_to_photo(self.fake_monet)
        self.fake_photo = self.G_monet_to_photo(self.real_monet) # monet - fake_photo - reconstructed_monet
        self.rec_monet = self.G_photo_to_monet(self.fake_photo)
    
    def backward_D_basic(self, netD, real, fake):
        # Базовая функция для обучения дискриминатора (отличие реальных изображений от сгенерированных)
        pred_real = netD(real)
        loss_D_real = self.criterion_gan(pred_real, torch.ones_like(pred_real))
        pred_fake = netD(fake.detach()) # Обязательное применение detach - останавливаем градиент через генератор
        loss_D_fake = self.criterion_gan(pred_fake, torch.zeros_like(pred_fake))
        
        # Общий loss дискриминатора (среднее между real и fake)
        loss_D = (loss_D_real + loss_D_fake) * 0.5
        loss_D.backward()
        return loss_D
    
    def backward_D_photo(self):
        # Обучение дискриминатора для фото, используем image pool для стабилизации
        fake_photo = self.fake_photo_buffer.query(self.fake_photo)
        self.loss_D_photo = self.backward_D_basic(self.D_photo, self.real_photo, fake_photo)
    
    def backward_D_monet(self):
        # Обучение дискриминатора для картин Моне
        fake_monet = self.fake_monet_buffer.query(self.fake_monet)
        self.loss_D_monet = self.backward_D_basic(self.D_monet, self.real_monet, fake_monet)
        
    def backward_G(self):
        # Обучение генераторов
        
        # Identity Loss - сохранение цветовой схемы исходных изображений
        idt_photo = self.G_monet_to_photo(self.real_photo)
        loss_idt_photo = self.criterion_identity(idt_photo, self.real_photo) * self.lambda_cycle * self.lambda_identity
        
        idt_monet = self.G_photo_to_monet(self.real_monet)
        loss_idt_monet = self.criterion_identity(idt_monet, self.real_monet) * self.lambda_cycle * self.lambda_identity
        
        # Adversarial Loss - обман дискриминаторов
        pred_fake_photo = self.D_photo(self.fake_photo)
        loss_G_monet_to_photo = self.criterion_gan(pred_fake_photo, torch.ones_like(pred_fake_photo))
        pred_fake_monet = self.D_monet(self.fake_monet)
        loss_G_photo_to_monet = self.criterion_gan(pred_fake_monet, torch.ones_like(pred_fake_monet))
        
        # Cycle Consistency Loss - гарантия того, что преобразование обратимо
        loss_cycle_photo = self.criterion_cycle(self.rec_photo, self.real_photo) * self.lambda_cycle
        loss_cycle_monet = self.criterion_cycle(self.rec_monet, self.real_monet) * self.lambda_cycle
        
        # Суммирование всех компонентов loss для генераторов
        self.loss_G = loss_G_photo_to_monet + loss_G_monet_to_photo + loss_cycle_photo + loss_cycle_monet + loss_idt_photo + loss_idt_monet
        self.loss_G.backward()
    
    def optimize_parameters(self):
        # Полный цикл обучения для одного батча
        
        # Forward pass - генерируем все необходимые изображения
        self.forward()

        # Обновление генераторов
        self.optimizer_G.zero_grad()  # Обнуляем градиенты
        self.backward_G()  # Вычисляем градиенты
        self.optimizer_G.step()  # Применяем обновление весов

        # Обновление дискриминатора для фото
        self.optimizer_D_photo.zero_grad()
        self.backward_D_photo()
        self.optimizer_D_photo.step()
        
        # Обновление дискриминатора для картин Моне
        self.optimizer_D_monet.zero_grad()
        self.backward_D_monet()
        self.optimizer_D_monet.step()


In [ ]:
# Основной цикл обучения CycleGAN

def train_cyclegan(model, dataloader, num_epochs=200, save_interval=50):
    # Перевод всех моделей в режим обучения
    model.G_photo_to_monet.train()
    model.G_monet_to_photo.train()
    model.D_photo.train()
    model.D_monet.train()
    
    for epoch in range(num_epochs):
        # Начальные значения для вычисления средних значений loss
        epoch_loss_G = 0
        epoch_loss_D_photo = 0
        epoch_loss_D_monet = 0
        
        progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for i, (photos, monets) in enumerate(progress_bar):
            # Установка входных данных и выполнение одного шага обучения
            model.set_input(photos, monets)
            model.optimize_parameters()
            
            # Накапливание losses для статистики
            epoch_loss_G += model.loss_G.item()
            epoch_loss_D_photo += model.loss_D_photo.item()
            epoch_loss_D_monet += model.loss_D_monet.item()
            
            # # Отображение текущих losses в progress bar
            # progress_bar.set_postfix({
            #     'Loss_G': f'{model.loss_G.item():.4f}',
            #     'Loss_D_Photo': f'{model.loss_D_photo.item():.4f}',
            #     'Loss_D_Monet': f'{model.loss_D_monet.item():.4f}'
            # })
        
        # Вычисление средних losses за эпоху
        avg_loss_G = epoch_loss_G / len(dataloader)
        avg_loss_D_photo = epoch_loss_D_photo / len(dataloader)
        avg_loss_D_monet = epoch_loss_D_monet / len(dataloader)
        
        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Loss_G: {avg_loss_G:.4f}, '
              f'Loss_D_Photo: {avg_loss_D_photo:.4f}, '
              f'Loss_D_Monet: {avg_loss_D_monet:.4f}') 
        
        # Логгирование метрик в ClearML
        logger.report_scalar(title='Loss Generator', series='Loss_G', value=avg_loss_G, iteration=epoch)
        logger.report_scalar(title='Loss Discriminator Photo', series='Loss_D_Photo', value=avg_loss_D_photo, iteration=epoch)
        logger.report_scalar(title='Loss Discriminator Monet', series='Loss_D_Monet', value=avg_loss_D_monet, iteration=epoch)
        
        # Периодическое сохранение checkpoint для возможности продолжения обучения
        if (epoch + 1) % save_interval == 0:
            checkpoint_path = f'cyclegan_checkpoint_epoch_{epoch+1}.pt'
            
            torch.save({
                'epoch': epoch,
                'G_photo_to_monet': model.G_photo_to_monet.state_dict(),
                'G_monet_to_photo': model.G_monet_to_photo.state_dict(),
                'D_photo': model.D_photo.state_dict(),
                'D_monet': model.D_monet.state_dict(),
                'optimizer_G': model.optimizer_G.state_dict(),
                'optimizer_D_photo': model.optimizer_D_photo.state_dict(),
                'optimizer_D_monet': model.optimizer_D_monet.state_dict(),
            }, checkpoint_path)
            print(f'Модель сохранена на эпохе {epoch+1}')
            
            # Загружаем в ClearML с метаданными
            task.upload_artifact(
                name=f'checkpoint_epoch_{epoch+1}',
                artifact_object=checkpoint_path,
                metadata={
                    'epoch': epoch+1,
                    'avg_loss_G': float(avg_loss_G),
                    'avg_loss_D_photo': float(avg_loss_D_photo),
                    'avg_loss_D_monet': float(avg_loss_D_monet),
                    'learning_rate': model.optimizer_G.param_groups[0]['lr']
                }
            )
            
        # Обновление learning rate согласно scheduler
        model.scheduler_G.step()
        model.scheduler_D_photo.step()
        model.scheduler_D_monet.step()


In [ ]:
# Вспомогательная функция для денормализации и конвертации тензора в изображение
def denorm_tensor_to_pil(img_tensor):
    t = img_tensor.clone().cpu()
    t = (t + 1.0) / 2.0  # Денормализация из [-1, 1] в [0, 1]
    t = t.clamp(0,1)  # Обрезка значения для безопасности
    t = (t * 255).byte()  # Конвертация в [0, 255]
    t = t.permute(1,2,0).numpy()  # CHW в HWC  
    return Image.fromarray(t) #Создание изображения из numpy массива

# Функция для генерации всех изображений в стиле Моне и сохранения в zip (обработка батчами для эффективности на GPU)
def generate_monet_images(model, photo_dir, zip_name='monet_generated.zip', device='cpu', batch_size=8):
    # Модели в режиме train (т.к. InstanceNorm)
    model.G_photo_to_monet.train()
    model.G_monet_to_photo.train()
    model.D_photo.train()
    model.D_monet.train()
    
    # Подготовка данных для генерации
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3), 
    ])
    
    photo_files = [f for f in os.listdir(photo_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    total_files = len(photo_files)
    generated_count = 0
    
    print(f"Начинаю генерацию {total_files} изображений батчами по {batch_size}...")
  
    # Создание zip файла для submission
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED, compresslevel=1) as zipf:
        with torch.no_grad():  # Отключение вычисления градиентов
            for batch_start in tqdm(range(0, total_files, batch_size), desc='Обработка батчей'):
                batch_end = min(batch_start + batch_size, total_files)
                batch_files = photo_files[batch_start:batch_end]
                
                # Формирование батча изображений
                batch_images = []
                valid_files = []
                
                for photo_file in batch_files:
                    try:
                        photo_path = os.path.join(photo_dir, photo_file)
                        img = Image.open(photo_path).convert('RGB')
                        img_tensor = transform(img)
                        batch_images.append(img_tensor)
                        valid_files.append(photo_file)
                    except Exception as e:
                        print(f'Ошибка загрузки {photo_file}: {e}')
                        continue
                
                if not batch_images:
                    continue
                
                # Объединение списка тензоров в один батч
                batch_tensor = torch.stack(batch_images).to(device)
                
                # Генерация изображений в стиле Моне
                if hasattr(model, 'G_photo_to_monet'):
                    generated_batch = model.G_photo_to_monet(batch_tensor)
                else:
                    generated_batch = model(batch_tensor)
                
                # Сохранение каждого изображения из батча
                for i, photo_file in enumerate(valid_files):
                    try:
                        out_tensor = generated_batch[i]
                        # Денормализация из [-1, 1] в [0, 1]
                        out_tensor = (out_tensor + 1.0) / 2.0
                        out_tensor = out_tensor.clamp(0, 1)
                        
                        # Конвертация в PIL изображение
                        out_tensor = (out_tensor * 255).byte()
                        out_tensor = out_tensor.permute(1, 2, 0).cpu().numpy()
                        pil_img = Image.fromarray(out_tensor)
                        
                        # Сохранение в памяти как JPEG
                        img_bytes = io.BytesIO()
                        pil_img.save(img_bytes, format='JPEG', quality=85, optimize=True)
                        img_bytes.seek(0)
                        
                        # Добавление в zip архив с правильным именем для submission
                        out_name = os.path.splitext(photo_file)[0] + '_monet.jpg'
                        zipf.writestr(out_name, img_bytes.getvalue())
                        generated_count += 1
                        
                    except Exception as e:
                        print(f'Ошибка обработки {photo_file}: {e}')
                        continue
                
                # Очистка памяти для предотвращения OOM на GPU
                del batch_tensor, generated_batch, batch_images
                torch.cuda.empty_cache() if device == 'cuda' else None
    
    return generated_count, zip_name


In [ ]:
# Настройка устройства для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используется устройство: {device}')

# Пути к данным 
monet_dir = '/kaggle/input/gan-getting-started/monet_jpg'  
photo_dir = '/kaggle/input/gan-getting-started/photo_jpg'  

# Проверка наличия данных
if not os.path.exists(monet_dir):
    print(f'ОШИБКА: Каталог {monet_dir} не найден!')
if not os.path.exists(photo_dir):
    print(f'ОШИБКА: Каталог {photo_dir} не найден!')

# Подготовка данных
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

if os.path.exists(monet_dir) and os.path.exists(photo_dir):
    # Создание датасета и dataloader
    dataset = MonetPhotoDataset(monet_dir, photo_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=6, shuffle=True, num_workers=4, pin_memory=True)
    print(f'Датасет создан: {len(dataset)} пар изображений')
    
    # Инициализация модели CycleGAN
    model = CycleGAN(device=device)
    print('Модель CycleGAN создана')
    
    # Подсчет параметров для понимания размера модели
    total_params = sum(p.numel() for p in model.G_photo_to_monet.parameters())
    print(f'Количество параметров генератора: {total_params:,}')
else:
    print('Создание датасета пропущено из-за отсутствующих каталогов')


In [ ]:
# Гиперпараметры обучения
NUM_EPOCHS = 201  
SAVE_INTERVAL = 50  

# Запуск процесса обучения
train_cyclegan(model, dataloader, num_epochs=NUM_EPOCHS, save_interval=SAVE_INTERVAL)


In [ ]:
# ID задачи, где хранится артефакт
task_id = 'bae9d2d4c61349b7b8ce92ae8000ae7b'

# Создаём пустой объект CycleGAN (такой же, как при обучении)
model = CycleGAN(device=device)

# Загружаем задачу
task = Task.get_task(task_id=task_id)

# Смотрим, какие артефакты есть у задачи
print('Доступные артефакты:', list(task.artifacts.keys()))

# Берём нужный артефакт (например, checkpoint)
artifact = task.artifacts['checkpoint_epoch_150']  # подставьте правильное имя 

# Скачиваем файл локально
local_path = artifact.get_local_copy()
print('Веса скачаны в:', local_path)

# Загружаем веса в модель
checkpoint = torch.load(local_path, map_location=device)
model.G_photo_to_monet.load_state_dict(checkpoint['G_photo_to_monet'])
model.G_monet_to_photo.load_state_dict(checkpoint['G_monet_to_photo'])
model.D_photo.load_state_dict(checkpoint['D_photo'])
model.D_monet.load_state_dict(checkpoint['D_monet'])

print('Веса успешно восстановлены')

In [ ]:
# Функция для быстрой визуализации результатов после обучения

def quick_generate_samples(model, photo_dir, num_samples=5):
    model.G_photo_to_monet.train()
    model.G_monet_to_photo.train()
    model.D_photo.train()
    model.D_monet.train()
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    # Выбор случайных фотографий для демонстрации
    photo_files = [f for f in os.listdir(photo_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    selected_photos = random.sample(photo_files, min(num_samples, len(photo_files)))
    
    print("Быстрая генерация примеров")
    
    # Визуализация примеров
    fig, axes = plt.subplots(2, num_samples, figsize=(20, 8))
    if num_samples == 1:
        axes = axes.reshape(2, 1)
    
    with torch.no_grad():  
        for i, photo_file in tqdm(enumerate(selected_photos)):
            try:
                photo_path = os.path.join(photo_dir, photo_file)
                original_img = Image.open(photo_path).convert('RGB')
                
                # Преобравание в тензор и генерирование стилизованного изображения
                input_tensor = transform(original_img).unsqueeze(0).to(device)
                generated_monet = model.G_photo_to_monet(input_tensor)
                # Денормализация для отображения
                generated_monet = (generated_monet * 0.5 + 0.5).clamp(0, 1)
                generated_img = transforms.ToPILImage()(generated_monet.squeeze(0).cpu())
                
                # Отображение оригинала и результата
                axes[0, i].imshow(original_img)
                axes[0, i].set_title('Оригинал')
                axes[0, i].axis('off')
                
                axes[1, i].imshow(generated_img)
                axes[1, i].set_title('Стиль Моне')
                axes[1, i].axis('off')
                
                print(f"Обработано: {photo_file}")
                
            except Exception as e:
                print(f"Ошибка: {photo_file} - {e}")
                axes[0, i].axis('off')
                axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Визуализация результатов на num_samples случайных примерах
if os.path.exists(photo_dir):
    quick_generate_samples(model, photo_dir, num_samples=5)
else:
    print("Каталог с фото не найден!")

In [ ]:
# Генерация всех изображений для submission в Kaggle

generated_count, zip_name = generate_monet_images(
    model=model,
    photo_dir=photo_dir,
    zip_name='images.zip',  
    device=device,
    batch_size=8  
)

print(f"Сгенерировано {generated_count} изображений в файле {zip_name}")


In [ ]:
# Расширенная функция для визуализации - сравнение оригинального фото, генерации и реальной картины Моне

def visualize_results(model, photo_dir, monet_dir, num_samples=4):
    model.G_photo_to_monet.train()
    model.G_monet_to_photo.train()
    model.D_photo.train()
    model.D_monet.train()
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    # Выбор случайных изображений
    photo_files = [f for f in os.listdir(photo_dir) if f.endswith('.jpg')]
    monet_files = [f for f in os.listdir(monet_dir) if f.endswith('.jpg')]
    
    selected_photos = random.sample(photo_files, min(num_samples, len(photo_files)))
    selected_monets = random.sample(monet_files, min(num_samples, len(monet_files)))
    
    # Создание сетки
    fig, axes = plt.subplots(3, num_samples, figsize=(15, 9))
    if num_samples == 1:
        axes = axes.reshape(3, 1)
    
    with torch.no_grad():
        for i in range(num_samples):
            # Загрузка и преобразование фотографии
            photo_path = os.path.join(photo_dir, selected_photos[i])
            photo_img = Image.open(photo_path).convert('RGB')
            photo_tensor = transform(photo_img).unsqueeze(0).to(model.device)
            
            # Генерирование изображения в стиле Моне
            fake_monet = model.G_photo_to_monet(photo_tensor)
            fake_monet = (fake_monet * 0.5 + 0.5).clamp(0, 1)  # Денормализация
            fake_monet_img = transforms.ToPILImage()(fake_monet.squeeze(0).cpu())
            
            # Загрузка реальной картины Моне для сравнения
            monet_path = os.path.join(monet_dir, selected_monets[i])
            monet_img = Image.open(monet_path).convert('RGB')
            
            # Отображение всех трех изображений в колонке
            axes[0, i].imshow(photo_img)
            axes[0, i].set_title('Оригинальная фотография')
            axes[0, i].axis('off')
            
            axes[1, i].imshow(fake_monet_img)
            axes[1, i].set_title('Сгенерированная картина')
            axes[1, i].axis('off')
            
            axes[2, i].imshow(monet_img)
            axes[2, i].set_title('Оригинальная картина Моне')
            axes[2, i].axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Финальная визуализация для сравнения результатов модели с реальными картинами Моне

if os.path.exists(monet_dir) and os.path.exists(photo_dir):
    print('Примеры результатов.')
    visualize_results(model, photo_dir, monet_dir, num_samples=4)
else:
    print('Каталоги с данными не найдены. Проверьте пути к monet_dir и photo_dir.')



In [ ]:
task.close()